In [1]:
from googlefinance.client import get_price_data, get_prices_data
import pandas as pd

In [6]:
# Single stock
# param_AAPL = {
#     'q': "AAPL", # Stock symbol (ex: "AAPL")
#     'i': "86400", # Interval size in seconds ("86400" = 1 day intervals)
#     'x': "NASD", # Stock exchange symbol on which stock is traded (ex: "NASDAQ, NYSE", "NYSEARCA")
#     'p': "10Y" # Period (Ex: "1Y" = 1 year) from today
# }
start = pd.to_datetime('2012-01-01')
end = pd.to_datetime('2015-01-01')     
df = pd.DataFrame(get_price_data({'q': "AAPL", 'i': "86400", 'x': "NASD", 'p': "20Y" }))['Close'].loc[start : end]
df.index = pd.to_datetime(df.index.date)
df = pd.DataFrame(df.values, columns=['AAPL'])

In [7]:
# Multiple stocks
start = pd.to_datetime('2012-01-01')
end = pd.to_datetime('2014-12-31')

params_STOCKS = [{'q': "SBUX", 'x': "NASD",},
                              {'q': "AAPL", 'x': "NASD",},
                              {'q': "GS", 'x': "NYSE",},
                              {'q': "GILD", 'x': "NASD",}]
df = get_prices_data(params_STOCKS, "20Y")
df.index = pd.to_datetime(df.index)
df = df.loc[start:end]

df.head(2)

,SBUX_Open,SBUX_High,SBUX_Low,SBUX_Close,SBUX_Volume,AAPL_Open,AAPL_High,AAPL_Low,AAPL_Close,AAPL_Volume,GS_Open,GS_High,GS_Low,GS_Close,GS_Volume,GILD_Open,GILD_High,GILD_Low,GILD_Close,GILD_Volume
2012-01-04,23.425,23.52,22.64,22.642,12923554.0,58.49,58.93,58.43,58.75,75564699.0,92.97,96.43,92.83,95.36,8646487.0,20.73,21.00,20.68,20.93,19564110.0
2012-01-05,22.700,23.32,22.64,23.080,13892148.0,58.57,59.24,58.47,59.06,65061108.0,94.73,95.13,93.68,94.74,7407211.0,20.98,21.03,20.85,21.01,16235940.0


In [33]:
# params_INDEX = [
#     # S&P 500
#     {
#         'q': ".INX",
#         'x': "INDEXSP",
#     }
#]
def SPY_return(start, end):
    df = get_prices_data( [{'q': ".INX", 'x': "INDEXSP",}], "20Y")
    df.index = pd.to_datetime(df.index)
    df = pd.DataFrame(df['.INX_Close'].loc[start:end])
    df.rename(columns={".INX_Close":"SPY"}, inplace = True)
    return df

start = pd.to_datetime('2012-01-01')
end = pd.to_datetime('2014-12-31')     # Be cautious
df = SPY_return(start, end)
df.head(2)

,Close
2012-01-04,1277.06
2012-01-05,1277.30


In [18]:
def multi_returns(start, end, symbols):
    columns = []
    for symbol, market in symbols:
        columns.append(symbol)
        
    df = pd.DataFrame(columns=columns)
    
    for symbol, market in symbols:
        df[symbol] = pd.DataFrame(get_price_data({'q': symbol, 'i': "86400", 'x': market, 'p': "20Y" }))['Close'].loc[start : end]
    
    df.index = pd.to_datetime(df.index.date)
    return df
        
start = pd.to_datetime('2016-01-01')
end = pd.to_datetime('2016-02-01')     
df = multi_returns(start, end, [("SBUX","NASD"), ("AAPL","NASD")])
df.pct_change().head()

,SBUX,AAPL
2016-01-01,NaN,NaN
2016-01-05,-0.029485,0.000855
2016-01-06,0.006694,-0.025059
2016-01-07,-0.008866,-0.019570
2016-01-08,-0.024772,-0.042205


In [ ]:
# Four asset example of the covariance matrix.
from googlefinance.client import get_price_data, get_prices_data
import pandas as pd

start = pd.to_datetime('2016-01-01')
end = pd.to_datetime('2016-02-01')     

df = pd.DataFrame(columns=['SBUX', 'AAPL', 'GS', 'GILD'])
df['SBUX'] = pd.DataFrame(get_price_data({'q': "SBUX", 'i': "86400", 'x': "NASD", 'p': "20Y" }))['Close'].loc[start : end]
df['AAPL'] = pd.DataFrame(get_price_data({'q': "AAPL", 'i': "86400", 'x': "NASD", 'p': "20Y" }))['Close'].loc[start : end]
df['GS'] = pd.DataFrame(get_price_data({'q': "GS", 'i': "86400", 'x': "NYSE", 'p': "20Y" }))['Close'].loc[start : end]
df['GILD'] = pd.DataFrame(get_price_data({'q': 'GILD', 'i': "86400", 'x': "NASD", 'p': "20Y" }))['Close'].loc[start : end]
df.index = pd.to_datetime(df.index.date)

returns = df.pct_change()[1:]